In [1]:
# token 6f73ff55033b5b53e547192607a69efe96d25a54a5aff7a3

In [2]:
import mlflow
import pandas as pd
from mlflow import tracking
from mlflow.entities import ViewType
import yaml
import json
import numpy as np
import sys  # Import the sys module for stderr
import os
import time

In [3]:
# Define a custom function to convert NumPy objects to standard Python objects
def numpy_to_python(obj):
    if isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, np.generic):
        return np.asscalar(obj)
    return obj

# Set the MLflow tracking server URI
#tracking_uri = "http://127.0.0.1:5000"  # Replace with your MLflow server URI
tracking_uri = os.environ["MLFLOW_TRACKING_URI"]  # Replace with your MLflow server URI
print(f"connecting with: {tracking_uri}", file=sys.stderr)
tracking.set_tracking_uri(tracking_uri)

connecting with: http://server:5000


In [4]:
# Retrieve all experiments and create a dictionary to map experiment IDs to names
print(f"searching for experiments", file=sys.stderr)
experiment_id_to_name = {}
for exp in mlflow.search_experiments():
    if "_test" not in exp.name:
        print(f" - found {exp.name}", file=sys.stderr)
        experiment_id_to_name[exp.experiment_id] = exp.name

# Retrieve all experiments
all_experiments = list(experiment_id_to_name.keys())

searching for experiments
 - found unconditional-benchmark-BSDS300
 - found unconditional-hybrid-pre-trained-circles
 - found unconditional-hybrid-pre-trained-moons
 - found unconditional-hybrid-circles
 - found unconditional-hybrid-moons
 - found conditional-circles
 - found conditional-moons
 - found unconditional-circles
 - found unconditional-moons
 - found unconditional-moons


In [5]:
#list(experiment_id_to_name.items())[12]

In [6]:
ts = time.time()
print("downloading runs", file=sys.stderr)
all_runs = []
progress_counter = 0
for experiment_id in all_experiments:
    try:
        runs = mlflow.search_runs(
            experiment_ids=[experiment_id],
            run_view_type=ViewType.ALL,
            max_results=1000000
        )
        all_runs.append(runs)
        progress_counter += 1
        print(f"Downloaded {progress_counter}/{len(all_experiments)} experiments", file=sys.stderr)
        time.sleep(1) # to avoid to many requests error (and speed up download time??)
    except Exception as e:
        all_runs.append(None)
        progress_counter += 1
        print(f"skipping {progress_counter} {list(experiment_id_to_name.items())[progress_counter-1]}", file=sys.stderr)
        
te = time.time()
te-ts

downloading runs
Downloaded 1/10 experiments
Downloaded 2/10 experiments
Downloaded 3/10 experiments
Downloaded 4/10 experiments
Downloaded 5/10 experiments
Downloaded 6/10 experiments
Downloaded 7/10 experiments
Downloaded 8/10 experiments
Downloaded 9/10 experiments
Downloaded 10/10 experiments


98.36778235435486

In [7]:
#all_runs[15]
#experiment_id_to_name[all_experiments[14]]

In [8]:
df = pd.concat(all_runs)
df["experiment_name"] = df.apply(lambda row: experiment_id_to_name[row["experiment_id"]], axis=1)

df = df.dropna(axis=1, how='all')
df = df.sort_values(["metrics.val_loss"])

for c in df.columns:
    print(c)

run_id
experiment_id
status
artifact_uri
start_time
end_time
metrics.loss
metrics.val_loss
metrics.lr
params.fit_kwds.epochs
params.fit_kwds.reduce_lr_on_plateau
params.workers
params.steps_per_epoch
params.monitor
params.opt_epsilon
params.fit_kwds.batch_size
params.initial_epoch
params.experiment_name
params.fit_kwds.verbose
params.model_kwds.distribution_kwds.order
params.fit_kwds.lr_patience
params.log_file
params.opt_name
params.opt_learning_rate
params.model_kwds.distribution_kwds.clip_to_bernstein_domain
params.opt_beta_2
params.fit_kwds.monitor
params.use_multiprocessing
params.model_kwds.distribution_kwds.high
params.epochs
params.sample_weight
params.model_kwds.parameter_kwds.dtype
params.fit_kwds.early_stopping
params.min_delta
params.class_weight
params.model_kwds.distribution
params.validation_steps
params.seed
params.model_kwds.parameter_kwds.conditional
params.batch_size
params.shuffle
params.model_kwds.distribution_kwds.low
params.patience
params.validation_batch_size
p

In [9]:
# clean run names for easy aggregation and comparison later on
def remove_exp_from_run_name(row):
    exp_name = row["experiment_name"]
    words = exp_name.split("-")

    # Remove the last word by slicing the list of words
    words_without_last = words[:-1]

    # Join the remaining words back into a single string using "-" as the delimiter
    exp_name = "_".join(words_without_last)
    
    run_name = row["tags.mlflow.runName"]
    new_name = run_name.replace(f"{exp_name}_", "")
    return new_name

df["tags.mlflow.runName"] = df.apply(remove_exp_from_run_name, axis=1)

## Generate summarizing dataframe

In [10]:
grouped_df  = df.groupby(["experiment_name", "tags.mlflow.runName"])

# dipslay
grouped_df.apply(lambda r: r[:])

run_id  \
experiment_name     tags.mlflow.runName                                         
conditional-circles bernstein_flow      373  7fda6820ed6f4a97bab468c2cd397049   
                                        376  03f38a5a4a784c6ea2381a134d5301d9   
                                        352  8c25ee6e07f24fc08f42e051d44da21f   
                                        354  4610459880544b7993a0749702c40623   
                                        357  5bb960759d8f40579922c6eb0fada780   
...                                                                       ...   
unconditional-moons multivariate_normal 48   22981ce7547049b193fb1a537c89989e   
                                        53   b2adce5846c24ca3b7ac0aefd9fa3090   
                                        44   a6bc6c478f724e2099d9f044631eb6a4   
                                        46   0630bcb77fab49e3920c87a489c32ec2   
                                        45   692d3e4228f749d0a61797489537e5aa   

                                                  experiment_id    status  \
experiment_name     tags.mlflow.runName                                     
conditional-circles bernstein_flow      373  788739528573814156  FINISHED   
                                        376  788739528573814156  FINISHED   
                                        352  788739528573814156  FINISHED   
                                        354  788739528573814156  FINISHED   
                                        357  788739528573814156  FINISHED   
...                                                         ...       ...   
unconditional-moons multivariate_normal 48   266110308394071622    FAILED   
                                        53   266110308394071622    FAILED   
                                        44   266110308394071622    FAILED   
                                        46   266110308394071622    FAILED   
                                        45   266110308394071622    FAILED   

                                                                                  artifact_uri  \
experiment_name     tags.mlflow.runName                                                          
conditional-circles bernstein_flow      373  file:///mlruns/788739528573814156/7fda6820ed6f...   
                                        376  file:///mlruns/788739528573814156/03f38a5a4a78...   
                                        352  file:///mlruns/788739528573814156/8c25ee6e07f2...   
                                        354  file:///mlruns/788739528573814156/461045988054...   
                                        357  file:///mlruns/788739528573814156/5bb960759d8f...   
...                                                                                        ...   
unconditional-moons multivariate_normal 48   file:///mlruns/266110308394071622/22981ce75470...   
                                        53   file:///mlruns/266110308394071622/b2adce5846c2...   
                                        44   file:///mlruns/266110308394071622/a6bc6c478f72...   
                                        46   file:///mlruns/266110308394071622/0630bcb77fab...   
                                        45   file:///mlruns/266110308394071622/692d3e4228f7...   

                                                                  start_time  \
experiment_name     tags.mlflow.runName                                        
conditional-circles bernstein_flow      373 2023-09-21 05:47:13.794000+00:00   
                                        376 2023-09-21 05:41:02.927000+00:00   
                                        352 2023-09-21 06:24:40.656000+00:00   
                                        354 2023-09-21 06:12:46.061000+00:00   
                                        357 2023-09-21 06:05:18.518000+00:00   
...                                                                      ...   
unconditional-moons multivariate_normal 48  2023-09-25 18:23:57.938000+00:00   
                    

### Best validation losses


 vgl mit arxiv.org/abs/2102.03509

In [11]:
idx = grouped_df["metrics.val_loss"].idxmin()
df_best_runs = df.loc[idx]
pd.pivot_table(df_best_runs[["experiment_name", 
                             "tags.mlflow.runName", 
                             "metrics.val_loss", 
                             ]], values="metrics.val_loss", index="experiment_name", columns=["tags.mlflow.runName"])

tags.mlflow.runName,bernstein_flow,coupling_bernstein_flow,masked_autoregressive_bernstein_flow,multivariate_bernstein_flow,multivariate_normal
experiment_name,,,,,
conditional-circles,-0.740474,-1.599084,-1.550712,-0.742274,-0.174440
conditional-moons,-0.940692,-2.093925,-2.065629,-0.939990,-0.459779
unconditional-benchmark-BSDS300,-84.282656,NaN,NaN,NaN,NaN
unconditional-circles,-0.131999,-0.738034,-0.626582,-0.135143,0.038944
unconditional-hybrid-circles,NaN,-0.874623,NaN,NaN,NaN
unconditional-hybrid-moons,NaN,-1.295389,NaN,NaN,NaN
unconditional-hybrid-pre-trained-circles,NaN,-0.725174,NaN,NaN,NaN
unconditional-hybrid-pre-trained-moons,NaN,-1.096500,NaN,NaN,NaN
unconditional-moons,-0.111342,-1.309156,-1.343714,-0.201921,0.153492


### Best N by experiment alone

In [12]:
df_best_n = df
df_best_n["rank"] = df_best_n.groupby("experiment_name")["metrics.val_loss"].rank(ascending=True, method="first").replace(np.nan, 1.0).astype(int)

N_BEST_RUNS = 3
df_best_n = df_best_n[df_best_n["rank"] <= N_BEST_RUNS]
df_best_n.pivot_table(values=df_best_n[["run_id","tags.mlflow.runName", "metrics.val_loss"]], index=["experiment_name","rank"], aggfunc="first")
#TODO: veränderte Parameter anzeigen

metrics.val_loss  \
experiment_name                          rank                     
conditional-circles                      1            -1.625391   
                                         2            -1.624483   
                                         3            -1.624059   
conditional-moons                        1            -2.118887   
                                         2            -2.118887   
                                         3            -2.116150   
unconditional-benchmark-BSDS300          1           -84.676888   
                                         2           -84.676888   
                                         3           -84.676888   
unconditional-circles                    1            -0.738035   
                                         2            -0.738035   
                                         3            -0.738035   
unconditional-hybrid-circles             1            -0.945502   
                                         2            -0.930065   
                                         3            -0.923051   
unconditional-hybrid-moons               1            -1.460489   
                                         2            -1.448670   
                                         3            -1.445103   
unconditional-hybrid-pre-trained-circles 1            -0.822808   
                                         2            -0.819657   
                                         3            -0.818552   
unconditional-hybrid-pre-trained-moons   1            -1.331302   
                                         2            -1.313770   
                                         3            -1.313566   
unconditional-moons                      1            -1.391217   
                                         2            -1.391217   
                                         3            -1.383733   

                                                                         run_id  \
experiment_name                          rank                                     
conditional-circles                      1     a3af65aba3d94abe87edee406d70f420   
                                         2     b12f518b850348f6bdb503f1f1e5e853   
                                         3     25213cf1e8b544eca0e61fb14930f3ed   
conditional-moons                        1     55d8cf321e6f4655bc44d4f7d5f1db7f   
                                         2     b5f3ae6d2ce44122969e8ca3b4a0496e   
                                         3     b8a64cc575664a2299eaf5a5ac7b41f1   
unconditional-benchmark-BSDS300          1     ffdb1241240d456e897d102469a5fbc2   
                                         2     48a954d441294befa6d2cf55122b0254   
                                         3     61c96050617b4c6395df3ee1d45ca8e8   
unconditional-circles                    1     c4c4fdf5248444e18dc5723414722d13   
                                         2     2d29fdcaacbe4755ab346c7d3484270a   
                                         3     939a20fba06b414eb03404d18e347ae6   
unconditional-hybrid-circles             1     258c09f2f3254530a6299d6732b7a5ff   
                                         2     3dc6fa17e2bc4a52b4578c2cd1954150   
                                         3     4fd61cebcaee43c2a620fa19ba35ed26   
unconditional-hybrid-moons               1     3df6600fc61d40539d035aa31c7ac19b   
                                         2     a8beb1fad4754cd79057bae077dfdf51   
                                         3     fca7d7f02f774e5d9ad8509a50a83844   
unconditional-hybrid-pre-trained-circles 1     38007e1610ca41719c93d87a09bf8a50   
                                         2     17eb3bec44eb436897df246441026182   
                                         3     1bc493576bdb4939aa38cb81477e4ef2   
unconditional-hybrid-pre-trained-moons   1     bb271ffb8933497eb1fe620825336123   
                                         2     a57a62129ff54ce6ae248418aa04d2ff   
                            

### Best N runs for each experiment by runName

In [13]:
df_best_n = df
df_best_n["rank"] = grouped_df["metrics.val_loss"].rank(ascending=True, method="first").replace(np.nan, 1.0).astype(int)

N_BEST_RUNS = 3
df_best_n = df_best_n[df_best_n["rank"] <= N_BEST_RUNS]
pd.options.display.max_rows = 100
table = df_best_n.pivot_table(values=df_best_n[["run_id", "metrics.val_loss","params.fit_kwds.batch_size", 
                             "params.fit_kwds.learning_rate", 
                             "params.fit_kwds.lr_patience", 
                             "params.model_kwds.distribution_kwds.order", 
                             "params.model_kwds.parameter_kwds.hidden_units"]], index=["experiment_name", "tags.mlflow.runName","rank"], aggfunc="first")
table

metrics.val_loss  \
experiment_name                          tags.mlflow.runName                  rank                     
conditional-circles                      bernstein_flow                       1            -0.744586   
                                                                              2            -0.744586   
                                                                              3            -0.742992   
                                         coupling_bernstein_flow              1            -1.625391   
                                                                              2            -1.624483   
                                                                              3            -1.624059   
                                         masked_autoregressive_bernstein_flow 1            -1.584214   
                                                                              2            -1.584214   
                                                                              3            -1.575803   
                                         multivariate_bernstein_flow          1            -0.742924   
                                                                              2            -0.742924   
                                                                              3            -0.742576   
                                         multivariate_normal                  1            -0.174571   
                                                                              2            -0.174504   
                                                                              3            -0.174431   
conditional-moons                        bernstein_flow                       1            -0.942335   
                                                                              2            -0.942335   
                                                                              3            -0.942064   
                                         coupling_bernstein_flow              1            -2.111921   
                                                                              2            -2.108145   
                                                                              3            -2.107362   
                                         masked_autoregressive_bernstein_flow 1            -2.118887   
                                                                              2            -2.118887   
                                                                              3            -2.116150   
                                         multivariate_bernstein_flow          1            -0.941576   
                                                                              2            -0.941576   
                                                                              3            -0.941152   
                                         multivariate_normal                  1            -0.459913   
                                                                              2            -0.459654   
                                                                              3            -0.459644   
unconditional-benchmark-BSDS300          bernstein_flow                       1           -84.676888   
                                                                              2           -84.676888   
                                                                              3           -84.676888   
unconditional-circles                    bernstein_flow                       1            -0.134674   
                                                                              2            -0.134284   
                                                                              3            -0.134106   
                                         coupling_bernstein_flow              1            -0.738035   
                            

In [14]:
table.to_excel("table.xlsx")